In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
response = requests.get("https://marvelcinematicuniverse.fandom.com/wiki/Category:Characters?from=Z")
html_page = response.text

In [4]:

next_link = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Characters"
# next_link = html_tree.find("a", class_="category-page__pagination-next wds-button wds-is-secondary")

characters_link_list = []
while next_link != None:
    response = requests.get(next_link)
    html_page = response.text
    html_tree = BeautifulSoup(html_page)
    results = html_tree.find_all("a", class_="category-page__member-link")
    next_link = html_tree.find(
        "a", class_="category-page__pagination-next wds-button wds-is-secondary")
    next_link = next_link["href"] if next_link != None else None

    for result in results:
        characters_link_list.append(
            [result.text, "https://marvelcinematicuniverse.fandom.com"+result["href"]])


In [5]:
import pandas as pd
lista_personaggi = pd.DataFrame(characters_link_list, columns=["Nome", "Link"])
lista_personaggi["Biografia"] = None

In [17]:
lista_personaggi.head()

,Nome,Link,Biografia
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,None
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,None
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,None
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,None
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,None


In [9]:
from tqdm import tqdm
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
    character_page_response = requests.get(row["Link"])
    character_page_tree = BeautifulSoup(character_page_response.text)
    biography_list = []
    biography_initial = character_page_tree.find_all(id="Biography")
    description_ended = False
    if(biography_initial):
        next_span = biography_initial[0].parent.find_next()

        while not description_ended:
            if next_span.name != "a" and next_span.name != "sup":
                biography_list.append(next_span.text)

            next_span = next_span.find_next()
            if next_span.name == "h2":
                description_ended = True
            biography_list = list(dict.fromkeys(biography_list))
            lista_personaggi.at[idx, "Biografia"] = biography_list
    # biography_list.append(biography_initial.find_next("span"))


  1%|          | 25/3746 [00:04<10:30,  5.90it/s]


KeyboardInterrupt: 